[Pwsh 7 adds a bunch of new operators](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_operators?view=powershell-7.3#null-coalescing-operator-), including `Null-coalescing` 

You need at least Pwsh 7+ to run the examples

In [1]:
# The docs start with this syntax:
$list = $Null
${list}?[0]

# I find it cleaner to use:
($list)?[0]

#Example: 
$samples = 'foo', 234, (Get-Date), $null, ''

'# Example that breaks'
$samples | Foreach-Object { 
    $_.GetType()     
} 
'# simple coerce to null'
$samples | Foreach-Object { 
    ($_).GetType()     
}

# Example that breaks

IsPublic IsSerial Name                                     BaseType
-------- -------- ----                                     --------
True     True     String                                   System.Object
True     True     Int32                                    System.ValueType
True     True     DateTime                                 System.ValueType
InvalidOperation: 
Line |
  13 |      $_.GetType()
     |      ~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
True     True     String                                   System.Object
# simple coerce to null
True     True     String                                   System.Object
True     True     Int32                                    System.ValueType
True     True     DateTime                                 System.ValueType
InvalidOperation: 
Line |
  17 |      ($_).GetType()
     |      ~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
True     True   

In [2]:
$samples = 'foo', 234, (get-date), $null, '' 

function shortenTypeNames {
    process {
        # expects types or nulls
        if($null -eq $_) { '[<trueNull>]' }
        
        ($_)?.GetType().Name ?? '<nullable>'
        | Join-String -op '[' -os ']'

    }
}


$samples | ShortenTypeNames

[String]
[Int32]
[DateTime]
[<trueNull>]
[<nullable>]
[String]


In [3]:

$testId = 0 ; $samples3 = '', 'bob', "sdf`n", $null, "`n"

function testNewUser {
    # very nested / messy code, to make notebooks shorter
    process { 
        $t = $_ # t is the [t]arget abbreviated
        [pscustomobject]@{ 
            Id           = $testId++
            Name1        = ($t)?.ToString()      ?? '<anonymousA>'
            Name2        = ($t)?.ToString().Name ?? '<anonymousB>'
            
            Name3        = if ($t) { 
                ($t)?.ToString().Name ?? '<anonymousC>'
            } 

            Render       = ($t)?.ToString() -replace '\s+', '␠'

            TrueNull     = $null -eq $t 
            TrueEmptyStr = $t -is 'string' -and $t.length -eq 0
            IsBlank      = [string]::IsNullOrWhiteSpace( $t )
            IsEmpty      = [string]::IsNullOrEmpty( $t )
        }
    } 
}

write-warning 'Notice the different results for Name2 and Name3'
$samples3
| testNewUser | Tee-Object -var 'q1' | ft -auto


Id Name1        Name2        Name3        Render TrueNull TrueEmptyStr IsBlank IsEmpty
-- -----        -----        -----        ------ -------- ------------ ------- -------
 0              <anonymousB>                        False         True    True    True
 1 bob          <anonymousB> <anonymousC> bob       False        False   False   False
 2 sdf…         <anonymousB> <anonymousC> sdf␠      False        False   False   False
 3 <anonymousA> <anonymousB>                         True        False    True    True
 4 …            <anonymousB> <anonymousC> ␠         False        False    True   False



In [4]:
# using the statement:
$a = gi .

# error: Invoking Method on Null
$a.GetType().Name

# then: this coerces to null
($fake)?.GetType().Name

write-warning 'warning: It doesn''t automatically protect you from invoking another function on $null'
($fake)?.GetType().Name.ToString()

write-warning "You *could* write it like this, it's getting to a point it should be another step"
(($fake)?.GetType().Name)?.ToString()

DirectoryInfo
InvalidOperation: 
Line |
  11 |  ($fake)?.GetType().Name.ToString()
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.


<!-- buffer dump



import-module pansies
import-module Ninmonkey.Console
# cannot get it to fallback in these expresssions
# even though it reliably does it in func. Maybe its
# that $foo didnt exist, but $_ was?
# or, maybe the () evaluation changes somethiing
hr
$null -eq ($Foo.Bar)?.FakeFunc().Name    # true
($Foo.Bar)?.FakeFunc().Name ?? 'fall'    # fall
hr
$null -eq ($Foo.Bar)?.FakeFunc().Name    #true
($Foo.Bar)?.FakeFunc() ?? 'fall'         # fall


($_)?.ToString().Name ?? '<anonymous>'

($_)?.ToString() ?? '<anonymous>'

hr
$null -eq ($_)?.ToString()          # true
($_)?.ToString() ?? 'fall'

$null -eq ($_)?.ToString().Name     # true
($_)?.ToString().Name ?? 'fall'











 -->